In [1]:
# import libaries
import pandas as pd
import googlemaps #for geocoding addresses
import geopandas as gpd
import numpy as np # for calculations

In [137]:
# import data
df = pd.read_csv('../data/john-dailey.csv')

In [3]:
# define google maps
gmaps = googlemaps.Client(key='HIDDEN')

### Clean the Data

In [138]:
# fill in NaN and NaT with NA
df = df.fillna('NA')

In [139]:
# clean zip field
df['zip'] = df['zip'].str.split('-').str[0]

In [140]:
# clean amount field
df['amount'] = df['amount'].astype('float')

In [152]:
# clean dataset of contributions
cleandf = df.query(" `Cont/Exp` == 'C' & `item type` == ['MO','CH','LO'] & `Amend. code` != ['D', 'A'] ").reset_index(drop=True)

### Data Summary

In [165]:
# unique contributors (estimate)
len(cleandf['name'].unique())

867

In [154]:
# total contributions
# subtract 1,000 to account for the refund removed during data cleaning
sum(df.query(" `Cont/Exp` == 'C' & `item type` == ['MO','CH','LO'] & `Amend. code` != ['D', 'A'] ")['amount']) - 1000

449683.44

### Geocode Data

In [155]:
# geocode address function
def geocode_address(dataframe):
# for each row in the dataframe
  for i, row in dataframe.iterrows():
    print(dataframe['name'][i], "donated", dataframe['amount'][i], "to John Dailey")
    # make sure there's an address
    geocode_result = gmaps.geocode(dataframe['address 1'][i] + dataframe['city'][i] + dataframe['state'][i] + dataframe['zip'][i])
    if geocode_result != []: 
        dataframe.loc[i, 'Latitude'] = geocode_result[0]['geometry']['location']['lat']
        dataframe.loc[i, 'Longitude'] = geocode_result[0]['geometry']['location']['lng']
    else:
        dataframe.loc[i, 'Latitude'] = 'NA'
        dataframe.loc[i, 'Longitude'] = 'NA'

  return dataframe

In [ ]:
# geocode the addresses
contributions_geocoded = geocode_address(cleandf)

In [172]:
# remove fields that don't have geometry
# dataframe length: 966 to 882
clean_contributions_geocoded = contributions_geocoded.query(" Latitude != 'NA' ")

In [173]:
# convert dataframe to geodataframe
contributionsgdf = gpd.GeoDataFrame(clean_contributions_geocoded, geometry=gpd.points_from_xy(clean_contributions_geocoded.Longitude, clean_contributions_geocoded.Latitude))

In [176]:
contributionsgdf.head(5)

,rpt code,line number,item date,Cont/Exp,name,address 1,address 2,city,state,zip,cont. type,occupation,item type,description,amount,Amend. code,lastname,Latitude,Longitude,geometry
0,9068,1,2021-03-19,C,Jon Ausman,2202 Woodlawn Drive,NA,Tallahassee,FL,32303,I,NA,CH,NA,100.0,NA,Ausman,30.467723,-84.303678,POINT (-84.30368 30.46772)
2,9068,3,2021-03-19,C,Courtney Whitis,403 Plantation Road,NA,Tallahassee,FL,32303,I,Political<br>Advisor,CH,NA,250.0,NA,Whitis,30.476478,-84.276723,POINT (-84.27672 30.47648)
3,9068,4,2021-03-19,C,Derek Whitis,403 Plantation Road,NA,Tallahassee,FL,32303,I,Political<br>Advisor,CH,NA,250.0,NA,Whitis,30.476478,-84.276723,POINT (-84.27672 30.47648)
4,9068,5,2021-03-19,C,Jim Wolf,1888 Witchtree Acres,NA,Tallahassee,FL,32312,I,Retired,CH,NA,150.0,NA,Wolf,30.548868,-84.24501,POINT (-84.24501 30.54887)
5,9068,6,2021-03-19,C,Lee Hinkle,824 Greenbrier Lane,NA,Tallahassee,FL,32308,I,Retired,CH,NA,250.0,NA,Hinkle,30.471874,-84.264279,POINT (-84.26428 30.47187)


In [177]:
# export geodataframe
contributionsgdf[['name', 'address 1', 'cont. type', 'amount', 'geometry']].to_file("../data/john-dailey.geojson", driver='GeoJSON')

### Brief Data Analysis

In [179]:
# how many donors live or operate in Tallahassee? (the ones who have geometry)
print(" {part}/{whole}, or {ratio}%, of John Dailey's donors live or operate in Tallahassee.".format(
    part=len(clean_contributions_geocoded.query(" city == 'Tallahassee'")),
    whole=len(clean_contributions_geocoded),
    ratio=100*round(len(clean_contributions_geocoded.query(" city == 'Tallahassee'"))/len(clean_contributions_geocoded),2)
))

 748/882, or 85.0%, of John Dailey's donors live or operate in Tallahassee.


In [180]:
print(" John Dailey's average contribution is ${average}.".format(average=round(sum(cleandf['amount'])/len(cleandf))))

 John Dailey's average contribution is $467.
